<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Alejandra Toro
- Nombre de alumno 2: Martín Torrico

### **Link de repositorio de GitHub:** [Insertar Repositorio](https://github.com/MartinTorricoP/Laboratorios_MDS7202)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [1]:
!pip install -qq xgboost optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.6 MB/s eta 0:00:00


# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


In [3]:
df['capacity'].value_counts()

,count
capacity,
330ml,2486
500ml,2485
1.5lt,2485


In [4]:
df.isnull().sum()

,0
id,0
date,0
city,0
lat,0
long,0
pop,0
shop,0
brand,0
container,0
capacity,0


## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np

#----------# 1
# Semilla
seed = 123

# Separamos los datos en train, validation y test
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=seed)
val_df, test_df = train_test_split(temp_df, test_size=(1/3), random_state=seed)

# Separar las características y la variable objetivo
X_train = train_df.drop(columns=['quantity'])
y_train = train_df['quantity']

X_val = val_df.drop(columns=['quantity'])
y_val = val_df['quantity']

X_test = test_df.drop(columns=['quantity'])
y_test = test_df['quantity']

#----------# 2
# Extraer día, mes y año de 'date'
def extract_date_features(df):
    df['date'] = pd.to_datetime(df['date'], format='%d/%m/%y')
    df['year'] = df['date'].dt.year.astype('category')
    df['month'] = df['date'].dt.month.astype('category')
    df['day'] = df['date'].dt.day.astype('category')
    return df

# Aplicar el transformer de fecha
X_train = extract_date_features(X_train)
X_val = extract_date_features(X_val)
X_test = extract_date_features(X_test)

#----------# 3
# Numéricas
num_col = ['pop', 'price', 'lat', 'long']  # que transformaremos
cat_col = ['city', 'shop', 'brand', 'container', 'capacity']  # que transformaremos
pass_cols = ['year', 'month', 'day'] # no transformaremos

# Transformer de numéricas
num_transformer = Pipeline(steps=[
  ('log_transformer', FunctionTransformer(np.log1p))
])

# Transformer de categóricas
cat_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Crear el ColumnTransformer
preprocessor = ColumnTransformer(
     transformers=[
         ('passthrough', 'passthrough', pass_cols),
        ('log', num_transformer, num_col),
        ('onehot', cat_transformer, cat_col)
    ]
)

# Ajustar el preprocessor al conjunto de datos de entrenamiento
preprocessor.set_output(transform='pandas')
preprocessor.fit(X_train)

# Transformar los datasets
X_train_transformed = preprocessor.transform(X_train)
X_val_transformed = preprocessor.transform(X_val)
X_test_transformed = preprocessor.transform(X_test)

# Renombrar las columnas para evitar los prefijos '__'
X_train_transformed.columns = [col.split('__')[-1] for col in X_train_transformed.columns]
X_val_transformed.columns = [col.split('__')[-1] for col in X_val_transformed.columns]
X_test_transformed.columns = [col.split('__')[-1] for col in X_test_transformed.columns]

#----------# 4
# Crear el Pipeline con los pasos anteriores y agregar el DummyRegressor
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor(strategy='mean'))
])

#----------# 5
# Entrenar el pipeline con los datos transformados
model_pipeline.fit(X_train, y_train)

# Predecir en el conjunto de validación
val_predictions = model_pipeline.predict(X_val)

# Calcular el MAE en el conjunto de validación
mae_dummy = mean_absolute_error(y_val, val_predictions)

# Mostrar el MAE
print(f"Mean Absolute Error del DummyRegressor en validación: {mae_dummy}")


Mean Absolute Error del DummyRegressor en validación: 13254.740836794412


Para el contexto del negocio, significa que si la demanda real de una bebida es, por ejemplo, de 20.000 bebidas de cierto tipo, se está prediciendo, en promedio, un valor que se desvía 13.255 unidades del valor real, ya sea por sobreestimando o subestimando ("20.000 +-
13.254").

In [24]:
#----------# 6
from xgboost import XGBRegressor

# Pipeline con XGBRegressor
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

# Convertir las columnas categóricas 'year', 'month' y 'day' a tipos enteros antes de ajustar XGBoost
X_train['year'] = X_train['year'].astype(int)
X_train['month'] = X_train['month'].astype(int)
X_train['day'] = X_train['day'].astype(int)

X_val['year'] = X_val['year'].astype(int)
X_val['month'] = X_val['month'].astype(int)
X_val['day'] = X_val['day'].astype(int)

X_test['year'] = X_test['year'].astype(int)
X_test['month'] = X_test['month'].astype(int)
X_test['day'] = X_test['day'].astype(int)

# Re-entrenar el pipeline con XGBRegressor
xgb_pipeline.fit(X_train, y_train)

# Predecir en el conjunto de validación usando XGBRegressor
val_predictions_xgb = xgb_pipeline.predict(X_val)

# Calcular el Error Absoluto Medio (MAE) en el conjunto de validación para XGBRegressor
mae_xgb = mean_absolute_error(y_val, val_predictions_xgb)

mae_xgb

2439.6775216730066

Podemos observar que el MAE disminuye aproximadamente de una demanda de 13.254 bebidas a una demanda de 2.440 bebidas lo cual es una diferencia bastante considerable.

Esto nos hace entender que xgb tiene una mejor capacidad predictiva en comparación al dummy regressor al desviarse menos del valor real.

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
#----------# 7
import pickle

# Guardar el modelo DummyRegressor en Google Drive
with open('/content/drive/MyDrive/dummy_regressor.pkl', 'wb') as dummy_file:
    pickle.dump(model_pipeline, dummy_file)

# Guardar el modelo XGBRegressor en Google Drive
with open('/content/drive/MyDrive/xgb_regressor.pkl', 'wb') as xgb_file:
    pickle.dump(xgb_pipeline, xgb_file)


In [9]:
import pickle

# Cargar el modelo DummyRegressor desde Google Drive
with open('/content/drive/MyDrive/dummy_regressor.pkl', 'rb') as dummy_file:
    dummy_model = pickle.load(dummy_file)

# Cargar el modelo XGBRegressor desde Google Drive
with open('/content/drive/MyDrive/xgb_regressor.pkl', 'rb') as xgb_file:
    xgb_model = pickle.load(xgb_file)


## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [37]:
# Inserte su código acá

X_train_transformed = preprocessor.fit_transform(X_train)
X_val_transformed = preprocessor.transform(X_val)

# convertir sin usar get feature names
X_train_df = pd.DataFrame(X_train_transformed, columns=X_train_transformed.columns if isinstance(X_train_transformed, pd.DataFrame) else preprocessor.transformers_[0][2] + num_col + cat_col)
X_val_df = pd.DataFrame(X_val_transformed, columns=X_train_df.columns)  # Mantener las mismas columnas en X_val_df

# definir constrainsts como diccionario
monotonic_constraints = {col: 0 for col in X_train_df.columns}
monotonic_constraints["log__price"] = -1  # relacion negativa price

xgb_regressor = XGBRegressor(monotone_constraints=monotonic_constraints)
xgb_regressor.fit(X_train_df, y_train)

y_val_pred_xgb = xgb_regressor.predict(X_val_df)
mae_xgb = mean_absolute_error(y_val, y_val_pred_xgb)

print(f"MAE en validación (XGBRegressor con constraint de monotonía): {mae_xgb}")

MAE en validación (XGBRegressor con constraint de monotonía): 2530.596669371699


In [38]:
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('xgb_regressor', XGBRegressor())
])

xgb_pipeline.named_steps['xgb_regressor'].set_params(monotone_constraints=monotonic_constraints)

# Entrenar el pipeline
xgb_pipeline.fit(X_train, y_train)

# Predicción en validación y cálculo del MAE
y_val_pred_xgb = xgb_pipeline.predict(X_val)
mae_xgb = mean_absolute_error(y_val, y_val_pred_xgb)
print(f'MAE en validación (XGBRegressor con constraint de monotonía): {mae_xgb}')

MAE en validación (XGBRegressor con constraint de monotonía): 2530.596669371699


In [ ]:
# Guardar el modelo XGBRegressor en Google Drive
with open('/content/drive/MyDrive/xgb_regressor.pkl', 'wb') as xgb_file:
    pickle.dump(xgb_pipeline, xgb_file)

**Respuesta**: Al comparar el MAE obtenido con el XGBRegressor anterior, se observa que este aumenta, lo que sugiere que el modelo sin restricciones se ajustaba mejor a los datos observados, aunque teoricamente se espera que la demanda disminuya cuando el precio aumenta, que es una relacion inversa comun en economia. Esto puede deberse a que existan otros factores en los datos que influyen en la demanda y no siempre siguen una relacion negativa con el precio.

## 1.3 Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [41]:
import optuna
from optuna.samplers import TPESampler
import time
optuna.logging.set_verbosity(optuna.logging.WARNING)
# Inserte su código acá

def objective(trial):
    # Inserte su código acá
    min_frequency = trial.suggest_float('onehot_min_frequency', 0.0, 1.0)

    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0, 1)
    reg_lambda = trial.suggest_float('reg_lambda', 0, 1)

    # Crear el preprocesador con el OneHotEncoder optimizado
    preprocessor = ColumnTransformer(
        transformers=[
            ('passthrough', 'passthrough', pass_cols),
            ('log', FunctionTransformer(np.log1p), num_col),
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, min_frequency=min_frequency), cat_col)
        ]
    ).set_output(transform='pandas')

    # Transformar los datos de entrenamiento y validación
    X_train_transformed = preprocessor.fit_transform(X_train)
    X_val_transformed = preprocessor.transform(X_val)

    # Convertir a DataFrame sin usar get_feature_names_out
    X_train_df = pd.DataFrame(X_train_transformed, columns=X_train_transformed.columns if isinstance(X_train_transformed, pd.DataFrame) else preprocessor.transformers_[0][2] + num_col + cat_col)
    X_val_df = pd.DataFrame(X_val_transformed, columns=X_train_df.columns)  # Mantener las mismas columnas en X_val_df

    # Definir monotonic_constraints como diccionario
    monotonic_constraints = {col: 0 for col in X_train_df.columns}
    if "log__price" in X_train_df.columns:
        monotonic_constraints["log__price"] = -1  # Relación negativa en 'price'

    # Crear el modelo XGBRegressor con los constraints aplicados como diccionario
    model = XGBRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_leaves=max_leaves,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        monotone_constraints=monotonic_constraints,
        random_state=123
    )

    # Entrenar el modelo directamente sobre los datos transformados
    start_time = time.time()
    model.fit(X_train_df, y_train)
    train_time = time.time() - start_time

    # Predecir en el conjunto de validación
    y_val_pred = model.predict(X_val_df)
    mae = mean_absolute_error(y_val, y_val_pred)

    # Almacenar el modelo si el entrenamiento fue menor a 5 minutos
    if train_time <= 300:  # 300 segundos = 5 minutos
        trial.set_user_attr("best_model", model)

    return mae

In [42]:
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=123))
study.optimize(objective, timeout=300)  # Limitar el tiempo total de entrenamiento

# Resultados
print(f'Número de trials: {len(study.trials)}')
print(f'Mejor MAE: {study.best_value}')
print(f'Mejores hiperparámetros: {study.best_params}')

Número de trials: 154
Mejor MAE: 1967.332922174977
Mejores hiperparámetros: {'onehot_min_frequency': 0.058416440731259844, 'learning_rate': 0.0909460097764785, 'n_estimators': 663, 'max_depth': 10, 'max_leaves': 98, 'min_child_weight': 3, 'reg_alpha': 0.298263291733435, 'reg_lambda': 0.36886775104326397}


**Respuesta**: Se observa que el modelo optimizado con optuna reduce significativamente en MAE, lo que indica una mejora en la precision del modelo. Esto se puede deber.a que los hiperparametros permiten al modelo ser mas flexible y precisio al capturar relaciones entre los datos, lo que permite una generalizacion sin depender estrictamente de una relacion inversa entre precio-cantidad.

* learning_rate: este hiperparametro controla la velocidad de aprendizaje, equilibrando la precision y el overfitting. Con un valor de 0.0909 cercano al limite superior del rango, permite que el modelo tenga un aprendizaje rapido y estable, adecuado para un modelo con n_estimators=663. El rango tiene sentido ya que valores bajos ayudan a evitar un ajuste demasiado rapido.
* n_estimators: representa el numero de arboles en un modelo, lo que indica que el modelo requiere una cantidad razonable para lograr un buen ajuste. El rango es adecuado, lo que permite tener desde modelos simples hasta mas complejos como el resultado obtenido.
* max_depth: define la profundidad maxima de cada arbol, y al tener 10 permite capturar patrones mas detallados en los datos sin ser tan complejo. El rango tambien hace sentido ya que profundidades menores ofrecen opciones para modelos msa simples
* max_leaves: limita el numero maximo de hojas en cada arbol, el modelo tiene un numero alto de divisiones (98), asi capturando relaciones complejas. El rango es amplo, permitiendo al modelo dividiones minimas como mas detalladas
* min_child_weight: controla el peso minimo  de obsevaciones en una hoja, con un valor igual a 3, establece un balance para evitar ramas poco representativas, reduciendo asi el overfitting.
* reg_alpha: es un parametro de regularizacion L1, que ayuda a evitar el overfitting penalizando la magnitud de los coeficientes.
* reg_lambda: es otro parametro de regularizacion L2 que tambien evita el overfitting pero penaliza de manera distinta.
* min_frequency: controla la frecuencia minima de una categoria para incluirla (hiperparametro de onehotencoder). Al obtener 0.0584, significa que se excluyen categorias con menos del 5.84% de aparicion, lo que ayuda a reducir el ruido.

In [ ]:
# Guardar el modelo XGBRegressor en Google Drive
best_model = study.best_trial.user_attrs["best_model"]

with open('/content/drive/MyDrive/mejor_modelo_xgb.pkl', 'wb') as xgb_file:
    pickle.dump(best_model, xgb_file)

## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

**Respuesta**: El prunning es una tecnica para detener automaticamente los trials que muestran bajo rendimiento durante las primeras etapas de entrenamiento, eliminandolas antes de terminar. Esto permite que se utilice el tiempo de optimizacion en trials mas prometedores, lo que reduce el tiempo de entrenamiento general y acelera el proceso de busqueda de hiperparamentros. Al utilizar esta tecnica, se deberia notar una reduccion en el tiempo total de optimizacion sin afectar la precision final del modelo, ya que los trials menos prometedores se descartan.

In [45]:
!pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 3.1 MB/s eta 0:00:00


In [46]:
from optuna.integration import XGBoostPruningCallback

In [53]:
# Inserte su código acá
optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective(trial):
    # Hiperparámetros de OneHotEncoder
    min_frequency = trial.suggest_float('onehot_min_frequency', 0.0, 1.0)

    # Hiperparámetros de XGBRegressor
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0, 1)
    reg_lambda = trial.suggest_float('reg_lambda', 0, 1)

    # Seguimiento de parámetros
    print(f"Trial {trial.number} - Parameters: "
          f"onehot_min_frequency={min_frequency}, learning_rate={learning_rate}, "
          f"n_estimators={n_estimators}, max_depth={max_depth}, max_leaves={max_leaves}, "
          f"min_child_weight={min_child_weight}, reg_alpha={reg_alpha}, reg_lambda={reg_lambda}")

    # Crear el preprocesador con el OneHotEncoder optimizado
    preprocessor = ColumnTransformer(
        transformers=[
            ('passthrough', 'passthrough', pass_cols),
            ('log', FunctionTransformer(np.log1p), num_col),
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, min_frequency=min_frequency), cat_col)
        ]
    ).set_output(transform='pandas')

    # Transformar los datos de entrenamiento y validación
    X_train_transformed = preprocessor.fit_transform(X_train)
    X_val_transformed = preprocessor.transform(X_val)

    # Convertir a DataFrame sin usar get_feature_names_out
    X_train_df = pd.DataFrame(X_train_transformed, columns=X_train_transformed.columns if isinstance(X_train_transformed, pd.DataFrame) else preprocessor.transformers_[0][2] + num_col + cat_col)
    X_val_df = pd.DataFrame(X_val_transformed, columns=X_train_df.columns)

    # Definir monotonic_constraints como diccionario
    monotonic_constraints = {col: 0 for col in X_train_df.columns}
    if "log__price" in X_train_df.columns:
        monotonic_constraints["log__price"] = -1

    # Crear el modelo XGBRegressor con los constraints aplicados y el callback de pruning
    model = XGBRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_leaves=max_leaves,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        monotone_constraints=monotonic_constraints,
        random_state=123,
        eval_metric="mae"  # Especificamos eval_metric aquí
    )

    # Callback de pruning
    pruning_callback = XGBoostPruningCallback(trial, "validation_0-mae")

    # Entrenar el modelo con early stopping y pruning
    start_time = time.time()
    model.fit(
        X_train_df, y_train,
        eval_set=[(X_val_df, y_val)],
        verbose=False
    )
    train_time = time.time() - start_time

    # Predecir y calcular el MAE
    y_val_pred = model.predict(X_val_df)
    mae = mean_absolute_error(y_val, y_val_pred)

    # Guardar el modelo si el tiempo de entrenamiento es menor a 5 minutos
    if train_time <= 300:  # 300 segundos = 5 minutos
        trial.set_user_attr("best_model", model)

    return mae


In [54]:
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=123))
study.optimize(objective, timeout=300, show_progress_bar=True)  # Limitar el tiempo total de entrenamiento

# Resultados
print(f'Número de trials: {len(study.trials)}')
print(f'Mejor MAE: {study.best_value}')
print(f'Mejores hiperparámetros: {study.best_params}')

   0%|          | 00:00/05:00

Trial 0 - Parameters: onehot_min_frequency=0.6964691855978616, learning_rate=0.02932779416008757, n_estimators=265, max_depth=7, max_leaves=72, min_child_weight=3, reg_alpha=0.9807641983846155, reg_lambda=0.6848297385848633
Trial 1 - Parameters: onehot_min_frequency=0.48093190148436094, learning_rate=0.0398196343012209, n_estimators=376, max_depth=8, max_leaves=44, min_child_weight=1, reg_alpha=0.3980442553304314, reg_lambda=0.7379954057320357
Trial 2 - Parameters: onehot_min_frequency=0.18249173045349998, learning_rate=0.018369723858601764, n_estimators=555, max_depth=7, max_leaves=64, min_child_weight=5, reg_alpha=0.7244553248606352, reg_lambda=0.6110235106775829
Trial 3 - Parameters: onehot_min_frequency=0.7224433825702216, learning_rate=0.03297293247146465, n_estimators=394, max_depth=4, max_leaves=29, min_child_weight=4, reg_alpha=0.09210493994507518, reg_lambda=0.43370117267952824
Trial 4 - Parameters: onehot_min_frequency=0.4308627633296438, learning_rate=0.049874824667380316, n

**Respuesta**: Se observa que el numero de trials es menor al anterior (98) y se obtiene un mejor mae, levemente mayor. Esto se puede deber a que con prunning se realizan menos trials, ya que como se menciono anteriormente, esta tecnica detiene los ensayos con bajo rendimiento, lo que reduce el tiempo y se concentra en trials con mejor rendimienot. Aunque el tiempo se optimizo, el MAE es solo un poco mayor, lo que se puede explicar en que algunos trials que se habrian completado sin prunnning y podrian haber dado un MAE menor, se detuvieron antes. Los hiperparametros son similares, pero dijieren en detalles como el max_depth y reg_alpha, lo que indica que este modelo no tuvo la misma capacidad de fitting. En resumen, esta tecninca optimiza el tiempo de busqueda sin afectar drasticamente la precision del modelo

In [ ]:
# Guardar el modelo XGBRegressor en Google Drive
best_model = study.best_trial.user_attrs["best_model"]

with open('/content/drive/MyDrive/mejor_modelo_xgb.pkl', 'wb') as xgb_file:
    pickle.dump(best_model, xgb_file)

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [56]:
# Inserte su código acá
import optuna.visualization as vis

# Historial de optimizacin
fig_optimization_history = vis.plot_optimization_history(study)
fig_optimization_history.show()

# coordenadas paralelas
fig_parallel_coordinates = vis.plot_parallel_coordinate(study)
fig_parallel_coordinates.show()

# importancia de hiperparámetros
fig_param_importance = vis.plot_param_importances(study)
fig_param_importance.show()


**Respuesta**: A partir de los graficos se observa que las mejoras del MAE se observan al rededor del trial 10, y a partir del 20 el modelo comienza a estabilizarse en torno a mejor rendimiento. En el grafico de coordenadas parlalas se obseva que los valores bajos de learning_rate (0.02 aprox) combinado con valores altos de max_depth y max_leaves estan asociados con mejores resultados. El grafico de importancia revela que onehot_min_frequency es el parametro mas influyente con diferencia, por lo que es determinante de la precision alcanzada del modelo.

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

| Modelo                     | MAE en Validación |
|----------------------------|-------------------|
| Baseline (DummyRegressor)  | 13254.74         |
| XGBoost sin Constraints    | 2439.67          |
| XGBoost con Constraints    | 2530.60          |
| XGBoost + Optuna           | 1967.33          |
| XGBoost + Optuna + Prunning| 1989.36          |

El mejor modelo (MAE mas bajo) es XGBoost+Optuna


In [ ]:
# Inserte su código acá
# Cargar el modelo DummyRegressor desde Google Drive
with open('/content/drive/MyDrive/mejor_modelo_xgb.pkl', 'rb') as best_model_file:
    best_model = pickle.load(best_model_file)

In [59]:
best_model = study.best_trial.user_attrs["best_model"]

# Transformar el conjunto de test utilizando el preprocesador
X_test_transformed = preprocessor.transform(X_test)

# Realizar predicciones en el conjunto de test
y_test_pred = best_model.predict(X_test_transformed)

# Calcular el MAE en el conjunto de test
mae_test = mean_absolute_error(y_test, y_test_pred)
mae_test

1935.5613742040885

**Respuesta:**

Se observa que se tiene un MAE menor al obtenido con validacion, que se pueden deber a la aleatoriedad de los datos. Esta diferencia también sugiere que el modelo está generalizando bien, ya que no se observa un gran incremento en el error al pasar de validación a test, lo que indica que no hay sobreajuste.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>